### toytree quartet functions (in progress)

In [1]:
import toytree
import itertools
import numpy as np

### get two random trees

In [2]:
t0 = toytree.rtree.unittree(10, seed=0)
t1 = toytree.rtree.unittree(10, seed=1)

In [3]:
toytree.mtree([t0, t1]).draw(ts='p', height=200);

<svg class="toyplot-canvas-Canvas" height="200.0px" id="t23e3b585a2eb4854bba17abf3a270b00" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 500.0 200.0" width="500.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9

### Plan for counting quartets (Illustrated below)

We will traverse the tree visiting every node in turn. At each node we will select the edge above it (towards the root) to be the focal 'split'. Each split can represent many possible quartets, where at least one tip can be sampled from each of the four edges leading from the split. In the example below, we are visiting node 12, and the focal split is shown in black. The four edges leaving this split are shown in red, pink, blue, and aqua. To get all quartets from this split we must sample all possible combinations of one sample from each colored set. 

In [4]:
t0.draw(
    ts='p',
    node_colors="lightgrey",
    edge_widths=3,
    edge_colors=t0.get_edge_values_mapped(
        {11: 'red', 3: 'pink', 4: 'blue', 18: 'aqua', 12: 'black'},
    ),
);

<svg class="toyplot-canvas-Canvas" height="260.0px" id="t08ee85451be948e9a0493fb599155bfb" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 350.0 260.0" width="350.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 r0 r1 r2 r3 r4 r5 r6 r7 r8 r9 0.0 0.5 1.0

### Example to sample tips from each quartet edge

In [5]:
# focal node
nidx = 12

# get all tips as a set
fullset = set(i for i in t0.get_tip_labels())

# get tips from each child of a given node
down0 = set(t0.idx_dict[nidx].children[0].get_leaf_names())
down1 = set(t0.idx_dict[nidx].children[1].get_leaf_names())
up0 = set(t0.idx_dict[nidx].up.get_leaf_names()) - down0 - down1
up1 = fullset - down0 - down1 - up0

print(down0)
print(down1)
print(up0)
print(up1)

{'r3'}
{'r0', 'r2', 'r1'}
{'r4'}
{'r8', 'r6', 'r7', 'r9', 'r5'}


### Example to get all quartet sets from sampled tips

In [6]:
set(itertools.product(down0, down1, up0, up1))

{('r3', 'r0', 'r4', 'r5'),
 ('r3', 'r0', 'r4', 'r6'),
 ('r3', 'r0', 'r4', 'r7'),
 ('r3', 'r0', 'r4', 'r8'),
 ('r3', 'r0', 'r4', 'r9'),
 ('r3', 'r1', 'r4', 'r5'),
 ('r3', 'r1', 'r4', 'r6'),
 ('r3', 'r1', 'r4', 'r7'),
 ('r3', 'r1', 'r4', 'r8'),
 ('r3', 'r1', 'r4', 'r9'),
 ('r3', 'r2', 'r4', 'r5'),
 ('r3', 'r2', 'r4', 'r6'),
 ('r3', 'r2', 'r4', 'r7'),
 ('r3', 'r2', 'r4', 'r8'),
 ('r3', 'r2', 'r4', 'r9')}

### Combine into a function

In [297]:
def get_quartets(ttre):
    
    # store all quartets in this SET
    qset = set([])
    
    # get a SET with all tips in the tree
    fullset = set(ttre.get_tip_labels())
    
    # get a SET of the descendants from each internal node
    for node in ttre.idx_dict.values():   

        # skip leaf nodes
        if not node.is_leaf():
            
            children = set(node.get_leaf_names())
            prod = itertools.product(
                itertools.combinations(children, 2),
                itertools.combinations(fullset - children, 2),
            )
            quartets = set([tuple(itertools.chain(*i)) for i in prod])
            qset = qset.union(quartets)

    # order tups in sets
    sorted_set = set()
    for qs in qset:
        if np.argmin(qs) > 1:
            tup = tuple(sorted(qs[2:]) + sorted(qs[:2]))
            sorted_set.add(tup)
        else:
            tup = tuple(sorted(qs[:2]) + sorted(qs[2:]))
            sorted_set.add(tup)            
    
    return sorted_set

In [291]:
get_quartets(t1)

{('r0', 'r1', 'r2', 'r3'),
 ('r0', 'r1', 'r2', 'r4'),
 ('r0', 'r1', 'r2', 'r5'),
 ('r0', 'r1', 'r3', 'r4'),
 ('r0', 'r1', 'r3', 'r5'),
 ('r0', 'r1', 'r4', 'r5'),
 ('r0', 'r2', 'r3', 'r4'),
 ('r0', 'r2', 'r3', 'r5'),
 ('r0', 'r2', 'r4', 'r5'),
 ('r0', 'r5', 'r3', 'r4'),
 ('r1', 'r2', 'r3', 'r4'),
 ('r1', 'r2', 'r3', 'r5'),
 ('r1', 'r2', 'r4', 'r5'),
 ('r1', 'r5', 'r3', 'r4'),
 ('r2', 'r5', 'r3', 'r4')}

### Compare quartet sets

In [292]:
q0 = get_quartets(t0)
q1 = get_quartets(t1)

In [294]:
# quartets that are in one tree but not the other
q0.symmetric_difference(q1)

{('r0', 'r3', 'r4', 'r5'),
 ('r0', 'r5', 'r3', 'r4'),
 ('r1', 'r3', 'r4', 'r5'),
 ('r1', 'r5', 'r3', 'r4'),
 ('r2', 'r3', 'r4', 'r5'),
 ('r2', 'r5', 'r3', 'r4')}

### what proportion of quartets are shared or different?